In [5]:
import os
if "ntbk" in os.getcwd():
    os.chdir("..")
print(os.getcwd())

import sys
sys.path.append(os.path.join(os.getcwd(), "otgnn"))


%load_ext autoreload
%autoreload 2

from otgnn.models import GCN
from otgnn.graph import MolGraph
from otgnn.utils import save_model, load_model

from mol_opt.data_mol_opt import MolOptDataset
from mol_opt.data_mol_opt import get_loader
from mol_opt.arguments import get_args
from mol_opt.train_mol_opt import main, get_latest_model
from mol_opt.ot_utils import compute_barycenter
from mol_opt.ot_utils import FGW 

from rdkit.Chem import MolFromSmiles
from rdkit.Chem.Draw import IPythonConsole

import torch
import numpy as np
import time

/home/octav/gitrepos/tum-thesis
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
model_type = "slot"
sys.argv = ["", "-cuda", "-model_type", model_type, "-one_batch_train"]
args = get_args()
args.n_epochs = 2000 
args.init_model = f"{model_type}10-onebatch"
args.output_dir = "mol_opt/output_{}".format(args.init_model)
args.tb_logs_dir = "mol_opt/logs_{}".format(args.init_model)
args.init_model = "{}10-onebatch-nonconstrained".format(model_type)
args.init_decoder_model = "{}_decode".format(args.init_model)

args.batch_size = 6
args.annealing_rate = 0.0002
args.connectivity_lambda = 1e-03
args.valency_lambda = 1e-04
args.euler_lambda = 1e-05
args.connectivity = False 
args.valency = False 
args.euler_characteristic_penalty = False
args.scale_lambdas = True
args.conn_penalty_function = "logdet" 
args

Namespace(N_transformer=6, agg_func='sum', annealing_rate=0.0002, batch_norm=False, batch_size=6, conn_penalty_function='logdet', connectivity=False, connectivity_hard=False, connectivity_lambda=0.001, cuda=True, device='cuda:0', dim_tangent_space=40, dropout_ffn=0.0, dropout_gcn=0.0, dropout_transformer=0.1, euler_characteristic_penalty=False, euler_lambda=1e-05, ffn_activation='LeakyReLU', init_decoder_model='slot10-onebatch-nonconstrained_decode', init_model='slot10-onebatch-nonconstrained', linear_out=False, model_type='slot', n_epochs=2000, n_ffn_hidden=100, n_ffn_transformer=100, n_heads_transformer=10, n_hidden=250, n_labels=1, n_layers=5, one_batch_train=True, ot_solver='emd', output_dir='mol_opt/output_slot10-onebatch', pc_hidden=100, pred_hidden=150, scale_lambdas=True, sinkhorn_entropy=0.1, sinkhorn_max_it=10000, task='qed', tb_logs_dir='mol_opt/logs_slot10-onebatch', valency=False, valency_hard=False, valency_lambda=0.0001)

In [7]:
train_data_loader = get_loader("iclr19-graph2graph/data/qed", "train_pairs", args.batch_size, False)
# val_data_loader = get_loader("iclr19-graph2graph/data/qed", "valid", 36, False)
val_data_loader = None

In [8]:
molopt, molopt_decoder = main(args, train_data_loader = train_data_loader, val_data_loader = val_data_loader)

No model slot10-onebatch-nonconstrained found in mol_opt/output_slot10-onebatch! Starting from scratch.
No model slot10-onebatch-nonconstrained_decode found in mol_opt/output_slot10-onebatch! Starting from scratch.
Epoch: 1
Epoch 1, train
 train_fgw:4.1152852
 train_conn_penalty:0.0000000
 train_val_penalty:0.0000000
 train_euler_penalty:0.0000000
 train_total:4.1152852
 train_similarity:0.0000000
 train_penlog:-100.0000000
Epoch duration: 0.24364757537841797
Epoch: 2
Epoch 2, train
 train_fgw:2.4471614
 train_conn_penalty:0.0000000
 train_val_penalty:0.0000000
 train_euler_penalty:0.0000000
 train_total:2.4471614
 train_similarity:0.0034014
 train_penlog:-8.9512133
Epoch duration: 0.2447199821472168
Epoch: 3
Epoch 3, train
 train_fgw:1.7843588
 train_conn_penalty:0.0000000
 train_val_penalty:0.0000000
 train_euler_penalty:0.0000000
 train_total:1.7843588
 train_similarity:0.0035461
 train_penlog:4.7770635
Epoch duration: 0.22771024703979492
Epoch: 4
Epoch 4, train
 train_fgw:3.2592589

Epoch 34, train
 train_fgw:0.5650556
 train_conn_penalty:0.0000000
 train_val_penalty:0.0000000
 train_euler_penalty:0.0000000
 train_total:0.5650556
 train_similarity:0.0065460
 train_penlog:1.9991393
Epoch duration: 0.23783206939697266
Epoch: 35
Epoch 35, train
 train_fgw:0.5624206
 train_conn_penalty:0.0000000
 train_val_penalty:0.0000000
 train_euler_penalty:0.0000000
 train_total:0.5624206
 train_similarity:0.0064780
 train_penlog:0.6574857
Epoch duration: 0.20958757400512695
Epoch: 36
Epoch 36, train
 train_fgw:0.6078960
 train_conn_penalty:0.0000000
 train_val_penalty:0.0000000
 train_euler_penalty:0.0000000
 train_total:0.6078960
 train_similarity:0.0065460
 train_penlog:0.5045001
Epoch duration: 0.2250504493713379
Epoch: 37
Epoch 37, train
 train_fgw:0.5438516
 train_conn_penalty:0.0000000
 train_val_penalty:0.0000000
 train_euler_penalty:0.0000000
 train_total:0.5438516
 train_similarity:0.0065460
 train_penlog:0.6170448
Epoch duration: 0.2482590675354004
Epoch: 38
Epoch 38, 

Epoch 68, train
 train_fgw:0.5552603
 train_conn_penalty:0.0000000
 train_val_penalty:0.0000000
 train_euler_penalty:0.0000000
 train_total:0.5552603
 train_similarity:0.0064878
 train_penlog:0.0113747
Epoch duration: 0.2546200752258301
Epoch: 69
Epoch 69, train
 train_fgw:0.5783392
 train_conn_penalty:0.0000000
 train_val_penalty:0.0000000
 train_euler_penalty:0.0000000
 train_total:0.5783392
 train_similarity:0.0066169
 train_penlog:0.9338322
Epoch duration: 0.20312786102294922
Epoch: 70
Epoch 70, train
 train_fgw:0.5318375
 train_conn_penalty:0.0000000
 train_val_penalty:0.0000000
 train_euler_penalty:0.0000000
 train_total:0.5318375
 train_similarity:0.0066693
 train_penlog:1.4500330
Epoch duration: 0.24931001663208008
Epoch: 71
Epoch 71, train
 train_fgw:0.5270828
 train_conn_penalty:0.0000000
 train_val_penalty:0.0000000
 train_euler_penalty:0.0000000
 train_total:0.5270828
 train_similarity:0.0064780
 train_penlog:-0.0406664
Epoch duration: 0.2987093925476074
Epoch: 72
Epoch 72,

Epoch 102, train
 train_fgw:0.5991629
 train_conn_penalty:0.0000000
 train_val_penalty:0.0000000
 train_euler_penalty:0.0000000
 train_total:0.5991629
 train_similarity:0.0312514
 train_penlog:1.4370986
Epoch duration: 0.4668905735015869
Epoch: 103
Epoch 103, train
 train_fgw:0.4836061
 train_conn_penalty:0.0000000
 train_val_penalty:0.0000000
 train_euler_penalty:0.0000000
 train_total:0.4836061
 train_similarity:0.0186992
 train_penlog:0.9628124
Epoch duration: 0.22680878639221191
Epoch: 104
Epoch 104, train
 train_fgw:0.5049436
 train_conn_penalty:0.0000000
 train_val_penalty:0.0000000
 train_euler_penalty:0.0000000
 train_total:0.5049436
 train_similarity:0.0213189
 train_penlog:0.8829484
Epoch duration: 0.2191929817199707
Epoch: 105
Epoch 105, train
 train_fgw:0.5340216
 train_conn_penalty:0.0000000
 train_val_penalty:0.0000000
 train_euler_penalty:0.0000000
 train_total:0.5340216
 train_similarity:0.0127189
 train_penlog:0.4694229
Epoch duration: 0.20628046989440918
Epoch: 106
Ep

Epoch 135, train
 train_fgw:0.4931642
 train_conn_penalty:0.0000000
 train_val_penalty:0.0000000
 train_euler_penalty:0.0000000
 train_total:0.4931642
 train_similarity:0.0127189
 train_penlog:0.6688062
Epoch duration: 0.2563188076019287
Epoch: 136
Epoch 136, train
 train_fgw:0.4981447
 train_conn_penalty:0.0000000
 train_val_penalty:0.0000000
 train_euler_penalty:0.0000000
 train_total:0.4981447
 train_similarity:0.0100027
 train_penlog:1.0440042
Epoch duration: 0.31499385833740234
Epoch: 137
Epoch 137, train
 train_fgw:0.4990728
 train_conn_penalty:0.0000000
 train_val_penalty:0.0000000
 train_euler_penalty:0.0000000
 train_total:0.4990728
 train_similarity:0.0127793
 train_penlog:-0.8357996
Epoch duration: 0.27661657333374023
Epoch: 138
Epoch 138, train
 train_fgw:0.4969530
 train_conn_penalty:0.0000000
 train_val_penalty:0.0000000
 train_euler_penalty:0.0000000
 train_total:0.4969530
 train_similarity:0.0160607
 train_penlog:0.8585857
Epoch duration: 0.23057341575622559
Epoch: 139


...

Epoch 1848, train
 train_fgw:0.5128754
 train_conn_penalty:0.0000000
 train_val_penalty:0.0000000
 train_euler_penalty:0.0000000
 train_total:0.5128754
 train_similarity:0.0292353
 train_penlog:-0.6743077
Epoch duration: 0.44207119941711426
Epoch: 1849
Epoch 1849, train
 train_fgw:0.4927627
 train_conn_penalty:0.0000000
 train_val_penalty:0.0000000
 train_euler_penalty:0.0000000
 train_total:0.4927627
 train_similarity:0.0090805
 train_penlog:-16.2083166
Epoch duration: 0.5018215179443359
Epoch: 1850
Epoch 1850, train
 train_fgw:0.5000109
 train_conn_penalty:0.0000000
 train_val_penalty:0.0000000
 train_euler_penalty:0.0000000
 train_total:0.5000109
 train_similarity:0.0061750
 train_penlog:2.1837875
Epoch duration: 0.2999391555786133
Epoch: 1851
Epoch 1851, train
 train_fgw:0.5034461
 train_conn_penalty:0.0000000
 train_val_penalty:0.0000000
 train_euler_penalty:0.0000000
 train_total:0.5034461
 train_similarity:0.0093780
 train_penlog:-14.5384893
Epoch duration: 0.36191630363464355
E

Epoch 1881, train
 train_fgw:0.4946912
 train_conn_penalty:0.0000000
 train_val_penalty:0.0000000
 train_euler_penalty:0.0000000
 train_total:0.4946912
 train_similarity:0.0155107
 train_penlog:0.0576765
Epoch duration: 0.3570590019226074
Epoch: 1882
Epoch 1882, train
 train_fgw:0.4826962
 train_conn_penalty:0.0000000
 train_val_penalty:0.0000000
 train_euler_penalty:0.0000000
 train_total:0.4826962
 train_similarity:0.0098140
 train_penlog:1.5306932
Epoch duration: 0.28188610076904297
Epoch: 1883
Epoch 1883, train
 train_fgw:0.4843208
 train_conn_penalty:0.0000000
 train_val_penalty:0.0000000
 train_euler_penalty:0.0000000
 train_total:0.4843208
 train_similarity:0.0226787
 train_penlog:-0.7746002
Epoch duration: 0.4351382255554199
Epoch: 1884
Epoch 1884, train
 train_fgw:0.5298664
 train_conn_penalty:0.0000000
 train_val_penalty:0.0000000
 train_euler_penalty:0.0000000
 train_total:0.5298664
 train_similarity:0.0117740
 train_penlog:-16.4518906
Epoch duration: 0.26304149627685547
Epo

Epoch 1914, train
 train_fgw:0.5013514
 train_conn_penalty:0.0000000
 train_val_penalty:0.0000000
 train_euler_penalty:0.0000000
 train_total:0.5013514
 train_similarity:0.0154932
 train_penlog:1.5410234
Epoch duration: 0.21630501747131348
Epoch: 1915
Epoch 1915, train
 train_fgw:0.5280976
 train_conn_penalty:0.0000000
 train_val_penalty:0.0000000
 train_euler_penalty:0.0000000
 train_total:0.5280976
 train_similarity:0.0182190
 train_penlog:2.0836116
Epoch duration: 0.30402302742004395
Epoch: 1916
Epoch 1916, train
 train_fgw:0.4874537
 train_conn_penalty:0.0000000
 train_val_penalty:0.0000000
 train_euler_penalty:0.0000000
 train_total:0.4874537
 train_similarity:0.0094904
 train_penlog:1.6999996
Epoch duration: 0.2336406707763672
Epoch: 1917
Epoch 1917, train
 train_fgw:0.5324999
 train_conn_penalty:0.0000000
 train_val_penalty:0.0000000
 train_euler_penalty:0.0000000
 train_total:0.5324999
 train_similarity:0.0144274
 train_penlog:0.4408294
Epoch duration: 0.42807793617248535
Epoch

Epoch 1947, train
 train_fgw:0.5264104
 train_conn_penalty:0.0000000
 train_val_penalty:0.0000000
 train_euler_penalty:0.0000000
 train_total:0.5264104
 train_similarity:0.0191506
 train_penlog:0.8344190
Epoch duration: 0.24976468086242676
Epoch: 1948
Epoch 1948, train
 train_fgw:0.4868527
 train_conn_penalty:0.0000000
 train_val_penalty:0.0000000
 train_euler_penalty:0.0000000
 train_total:0.4868527
 train_similarity:0.0300011
 train_penlog:-15.4995638
Epoch duration: 0.21565675735473633
Epoch: 1949
Epoch 1949, train
 train_fgw:0.4368591
 train_conn_penalty:0.0000000
 train_val_penalty:0.0000000
 train_euler_penalty:0.0000000
 train_total:0.4368591
 train_similarity:0.0249119
 train_penlog:1.0526478
Epoch duration: 0.2582111358642578
Epoch: 1950
Epoch 1950, train
 train_fgw:0.4675455
 train_conn_penalty:0.0000000
 train_val_penalty:0.0000000
 train_euler_penalty:0.0000000
 train_total:0.4675455
 train_similarity:0.0182339
 train_penlog:1.5375463
Epoch duration: 0.3522677421569824
Epoc

Epoch 1980, train
 train_fgw:0.4970760
 train_conn_penalty:0.0000000
 train_val_penalty:0.0000000
 train_euler_penalty:0.0000000
 train_total:0.4970760
 train_similarity:0.0195062
 train_penlog:2.5946970
Epoch duration: 0.5405256748199463
Epoch: 1981
Epoch 1981, train
 train_fgw:0.4701320
 train_conn_penalty:0.0000000
 train_val_penalty:0.0000000
 train_euler_penalty:0.0000000
 train_total:0.4701320
 train_similarity:0.0127793
 train_penlog:1.8323414
Epoch duration: 0.3230772018432617
Epoch: 1982
Epoch 1982, train
 train_fgw:0.5079169
 train_conn_penalty:0.0000000
 train_val_penalty:0.0000000
 train_euler_penalty:0.0000000
 train_total:0.5079169
 train_similarity:0.0155291
 train_penlog:2.1968427
Epoch duration: 0.3236064910888672
Epoch: 1983
Epoch 1983, train
 train_fgw:0.5122361
 train_conn_penalty:0.0000000
 train_val_penalty:0.0000000
 train_euler_penalty:0.0000000
 train_total:0.5122361
 train_similarity:0.0187946
 train_penlog:-15.2176820
Epoch duration: 0.4208223819732666
Epoch:

In [37]:
train_data_loader = get_loader("iclr19-graph2graph/data/qed", "train_pairs", args.batch_size, False)

for i in train_data_loader:
    X = (MolGraph(i[0]))
    Y = (MolGraph(i[1]))
    break

In [40]:
fgw_loss = FGW(alpha = 0.5)

In [45]:
x_embedding = molopt.forward(X)
yhat_logits = molopt_decoder.forward(x_embedding, X, Y)
yhat_labels = molopt_decoder.discretize_argmax(*yhat_logits)
# yhat_labels = molopt_decoder.discretize(*yhat_logits)
pred_pack = (yhat_labels, yhat_logits, Y.scope), Y 

target = Y.get_graph_outputs()
symbols_labels, charges_labels, bonds_labels = yhat_labels
symbols_logits, charges_logits, bonds_logits = yhat_logits

print (fgw_loss(*pred_pack, tau = 1).item()/args.batch_size)
pen_loss = pen(*pred_pack, model_iter)
print (pen_loss)
metrics.measure_batch(pred_pack[0], target)

0.22125347455342612


NameError: name 'pen' is not defined

In [69]:
molopt.project(molopt.GCN(X)[0], X)

tensor([[-0.0691,  0.0228, -0.1238,  ..., -0.2276, -0.1211,  0.0346],
        [ 0.0895, -0.0330,  0.3422,  ..., -0.3231, -0.2001,  0.1365],
        [ 0.2503,  0.0699,  0.1035,  ...,  0.0921, -0.1377, -0.1102],
        ...,
        [-0.1472,  0.1102, -0.0429,  ...,  0.1742, -0.0442,  0.2404],
        [-0.0445,  0.0704,  0.0658,  ...,  0.0397,  0.1655, -0.1985],
        [-0.3015, -0.0202, -0.1621,  ..., -0.0598, -0.0161,  0.1441]],
       device='cuda:0', grad_fn=<CopySlices>)

In [70]:
molopt.ref

tensor([[-0.8220, -0.4019,  0.8665,  ...,  1.7045, -0.1772, -0.3348],
        [ 0.0944, -1.2275, -1.7639,  ...,  0.9517,  0.2938, -1.2259],
        [ 0.3994, -1.5773, -0.9505,  ..., -1.6369,  0.4553,  0.9596],
        ...,
        [ 0.8281, -0.7141, -0.2793,  ..., -0.1753, -1.8184, -1.6099],
        [-0.5090,  0.3673, -0.5418,  ..., -0.0758, -0.3708,  1.5190],
        [-0.5584, -0.5709,  0.5864,  ...,  0.6412, -1.4830, -1.2610]],
       device='cuda:0', requires_grad=True)

In [ ]:
args.dim_tangent_space * args.pc_hidden